# Test on FLCHAIN

In [1]:
import numpy

In [2]:
import sklearn

In [3]:
import survwrap

In [4]:
survwrap.list_available_datasets()

('flchain', 'gbsg2', 'metabric', 'support')

In [5]:
mb_df = survwrap.get_data('flchain')
mb_df.dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6524 entries, 0 to 6523
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   age         6524 non-null   float64
 1   sex         6524 non-null   float64
 2   sample.yr   6524 non-null   int64  
 3   kappa       6524 non-null   float64
 4   lambda      6524 non-null   float64
 5   flc.grp     6524 non-null   int64  
 6   creatinine  6524 non-null   float64
 7   mgus        6524 non-null   float64
 8   time        6524 non-null   float64
 9   event       6524 non-null   float64
dtypes: float64(8), int64(2)
memory usage: 560.7 KB


In [6]:
X, y = mb_df.get_X_y()
X.shape, y.shape

((6524, 8), (6524,))

In [7]:
y[:10]

array([( True,   85.), ( True, 1281.), ( True,   69.), ( True,  115.),
       ( True, 1039.), ( True, 1355.), ( True, 2851.), ( True,  372.),
       ( True, 3309.), ( True, 1326.)],
      dtype=[('event', '?'), ('time', '<f8')])

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [8]:
from sklearn.preprocessing import StandardScaler, RobustScaler

In [9]:
X_train, X_test, y_train, y_test = survwrap.survival_train_test_split(X, y, rng_seed=2309)

In [10]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((4893, 8), (1631, 8))

In [11]:
survwrap.get_indicator(y).sum(), survwrap.get_indicator(y_train).sum(), survwrap.get_indicator(y_test).sum(),


(1962, 1472, 490)

## check possible dimensionality reduction

In [12]:
from sklearn.decomposition import PCA

In [13]:
pca= PCA(n_components=0.995, random_state=2308).fit(X_train)
print('PCA components:',pca.n_components_)

PCA components: 8


Massive (50%) reduction using PCA

In [14]:
## Stratified CV spliter for survival analysis

In [15]:
#from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

In [16]:
#testkf= RepeatedStratifiedKFold(n_splits=5,n_repeats=2,random_state=2307)
#for trn,tst in testkf.split(X_train, survwrap.get_indicator(y_train)):
#    print(trn,tst) 

# test coxnet

In [17]:
rng_seed=2401

In [18]:
coxnet = survwrap.CoxNet(rng_seed)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2401, alpha=None, l1_ratio=0.5)

In [19]:
coxnet.score(X_test, y_test)

0.7927079761717836

In [20]:
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [21]:
cox_grid = survwrap.CoxNet().get_parameter_grid()
cox_grid['l1_ratio'] = [0.5]
cox_grid

{'alpha': [0.001,
  0.003,
  0.005,
  0.008,
  0.01,
  0.02,
  0.03,
  0.04,
  0.05,
  0.06,
  0.07,
  0.08,
  0.09,
  0.1,
  0.15,
  0.2,
  0.3],
 'l1_ratio': [0.5]}

In [22]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = survwrap.optimize(survwrap.CoxNet(rng_seed), X_train, y_train, 
                                                                    user_grid=cox_grid, n_jobs=4)
opt_coxnet.score(X_test, y_test), opt_coxnet_params

(0.793126155601201, {'alpha': 0.02, 'l1_ratio': 0.5})

In [23]:
if opt_coxnet_search.scoring: print('Grullo!') 
else: print('OK')

OK


In [24]:
#opt_coxnet_search.cv_results_
survwrap.get_model_scores_df(opt_coxnet_search)

KeyError: 'rank_test_mean_fit_time'

# Test FastCPH

In [59]:
batch5 = int(X_train.shape[0] /20 ) 

In [60]:
fl_lasso=survwrap.FastCPH(rng_seed)
fl_lasso

FastCPH(rng_seed=2401, layer_sizes=[10, 10], tie_approximation='efron', lambda_seq=[0.001, 0.0010249999999999999, 0.001050625, 0.0010768906249999997, 0.0011038128906249996, 0.0011314082128906244, 0.00115969341821289, 0.0011886857536682123, 0.0012184028975099172, 0.0012488629699476653, 0.0012800845441963567, 0.0013120866578012656, 0.0013448888242462971, 0.0013785110448524542, 0.0014129738209737656, 0.0014482981664981096, 0.0014845056206605623, 0.001521618261177076, 0.001559658717706503, 0.0015986501856491654, 0.0016386164402903943, 0.0016795818512976542, 0.0017215713975800952, 0.0017646106825195975, 0.0018087259495825872, 0.0018539440983221518, 0.0019002927007802054, 0.0019478000182997105, 0.001996495018757203, 0.002046407394226133, 0.002097567579081786, 0.0021500067685588306, 0.002203756937772801, 0.0022588508612171212, 0.0023153221327475485, 0.0023732051860662373, 0.002432535315717893, 0.00249334869861084, 0.002555682416076111, 0.0026195744764780132, 0.0026850638383899635, 0.0027521904343497123, 0.0028209951952084546, 0.002891520075088666, 0.0029638080769658825, 0.0030379032788900293, 0.0031138508608622795, 0.0031916971323838364, 0.003271489560693432, 0.0033532767997107677, 0.0034371087197035363, 0.003523036437696125, 0.0036111123486385274, 0.0037013901573544903, 0.0037939249112883522, 0.003888773034070561, 0.003985992359922324, 0.004085642168920382, 0.004187783223143391, 0.0042924778037219755, 0.004399789748815024, 0.0045097844925354, 0.004622529104848784, 0.004738092332470004, 0.004856544640781753, 0.004977958256801297, 0.005102407213221329, 0.0052299673935518615, 0.005360716578390657, 0.005494734492850423, 0.005632102855171683, 0.005772905426550975, 0.005917228062214749, 0.006065158763770117, 0.0062167877328643695, 0.006372207426185978, 0.006531512611840627, 0.006694800427136643, 0.006862170437815058, 0.007033724698760434, 0.007209567816229444, 0.00738980701163518, 0.007574552186926058, 0.007763915991599209, 0.007958013891389188, 0.008156964238673917, 0.008360888344640766, 0.008569910553256782, 0.008784158317088201, 0.009003762275015407, 0.009228856331890791, 0.00945957774018806, 0.00969606718369276, 0.009938468863285078, 0.010186930584867204, 0.010441603849488884, 0.010702643945726105, 0.010970210044369256, 0.011244465295478486, 0.011525576927865449, 0.011813716351062082, 0.012109059259838635, 0.0124117857413346, 0.012722080384867962, 0.01304013239448966, 0.013366135704351901, 0.013700289096960697, 0.014042796324384713, 0.014393866232494331, 0.014753712888306687, 0.015122555710514353, 0.01550061960327721, 0.01588813509335914, 0.016285338470693116, 0.016692471932460443, 0.017109783730771953, 0.017537528324041252, 0.01797596653214228, 0.018425365695445835, 0.018885999837831978, 0.019358149833777778, 0.019842103579622218, 0.020338156169112772, 0.02084661007334059, 0.021367775325174105, 0.021901969708303454, 0.02244951895101104, 0.023010756924786314, 0.023586025847905968, 0.024175676494103616, 0.0247800684064562, 0.025399570116617608, 0.026034559369533045, 0.02668542335377137, 0.02735255893761565, 0.02803637291105604, 0.02873728223383244, 0.029455714289678244, 0.0301921071469202, 0.030946909825593203, 0.03172058257123303, 0.03251359713551385, 0.0333264370639017, 0.03415959799049924, 0.035013587940261715, 0.035888927638768255, 0.03678615082973745, 0.03770580460048089, 0.03864844971549291, 0.039614660958380225, 0.04060502748233973, 0.04162015316939822, 0.042660656998633174, 0.043727173423599, 0.04482035275918896, 0.045940861578168686, 0.0470893831176229, 0.04826661769556347, 0.04947328313795255, 0.05071011521640136, 0.051977868096811396, 0.053277314799231666, 0.054609247669212456, 0.05597447886094276, 0.05737384083246633, 0.05880818685327798, 0.060278391524609926, 0.06178535131272517, 0.0633299850955433, 0.06491323472293187, 0.06653606559100515, 0.06819946723078028, 0.06990445391154979, 0.07165206525933851, 0.07344336689082198, 0.07527945106309253, 0.07716143733966983, 0.07909047327316157, 0.0810677351049906, 0.0830944284826153

In [61]:
#fl_dsm.layer_sizes=[3]
#fl_dsm.learning_rate=0.01
fl_lasso_xplore=fl_lasso.fit(X_train,y_train)
fl_lasso_xplore.score(X_train,y_train)

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/lassonet/utils.py:28: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664817693763/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1550.)
  return torch.empty(output_size, device=input.device).scatter_reduce(


epoch: 0
loss: 8.593567848205566
epoch: 1
loss: 8.584389686584473
epoch: 2
loss: 8.575233459472656
epoch: 3
loss: 8.5661039352417
epoch: 4
loss: 8.556997299194336
epoch: 5
loss: 8.547910690307617
epoch: 6
loss: 8.538849830627441
epoch: 7
loss: 8.529813766479492
epoch: 8
loss: 8.520800590515137
epoch: 9
loss: 8.51181411743164
epoch: 10
loss: 8.502851486206055
epoch: 11
loss: 8.493910789489746
epoch: 12
loss: 8.484992027282715
epoch: 13
loss: 8.476089477539062
epoch: 14
loss: 8.46721076965332
epoch: 15
loss: 8.458356857299805
epoch: 16
loss: 8.449514389038086
epoch: 17
loss: 8.440692901611328
epoch: 18
loss: 8.431884765625
epoch: 19
loss: 8.42309284210205
epoch: 20
loss: 8.414307594299316
epoch: 21
loss: 8.40553092956543
epoch: 22
loss: 8.396761894226074
epoch: 23
loss: 8.387993812561035
epoch: 24
loss: 8.379231452941895
epoch: 25
loss: 8.370476722717285
epoch: 26
loss: 8.361723899841309
epoch: 27
loss: 8.35296630859375
epoch: 28
loss: 8.344212532043457
epoch: 29
loss: 8.33544921875
epoc

0.7942605159620818

In [63]:
fl_grid =fl_lasso.get_parameter_grid(max_width=X_train.shape[1])
fl_grid

{'layer_sizes': [[8], [8, 8], [8, 8, 8], [8, 8, 8, 8]]}

In [29]:
# Stratified CV
#opt_dsm, opt_dsm_params, opt_dsm_search = optimize(survwrap.DeepSurvivalMachines(rng_seed=2308),  X_train, y_train, n_jobs=8,
                                                  # user_grid=grid,cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=2308).split(X_train,survwrap.get_indicator(y_train)))
#opt_dsm.score(X_test, y_test), opt_dsm_params

In [37]:
# Stratified CV
fl_lasso_cv=survwrap.FastCPH(rng_seed)
fl_grid=fl_lasso_cv.get_parameter_grid(max_width=X_train.shape[1])
fl_grid

{'layer_sizes': [[8], [8, 8], [8, 8, 8], [8, 8, 8, 8]]}

In [39]:
opt_lasso, opt_lasso_params, opt_lasso_search = survwrap.optimize(fl_lasso_cv,  X_train, y_train, 
                                                                  user_grid=fl_grid,
                                                                 cv=survwrap.survival_crossval_splitter(X_train,y_train,
                                                                                                        n_repeats=1,n_splits=3),
                                                                 )

/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/lassonet/utils.py:28: UserWarning: scatter_reduce() is in beta and the API may change at any time. (Triggered internally at  /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664817693763/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:1550.)
  return torch.empty(output_size, device=input.device).scatter_reduce(


epoch: 0
loss: 7.848711013793945
epoch: 1
loss: 7.841729164123535
epoch: 2
loss: 7.8348798751831055
epoch: 3
loss: 7.828098297119141
epoch: 4
loss: 7.821382999420166
epoch: 5
loss: 7.814732551574707
epoch: 6
loss: 7.8081464767456055
epoch: 7
loss: 7.801629066467285
epoch: 8
loss: 7.795176029205322
epoch: 9
loss: 7.788789749145508
epoch: 10
loss: 7.782468318939209
epoch: 11
loss: 7.776213645935059
epoch: 12
loss: 7.770027160644531
epoch: 13
loss: 7.763906478881836
epoch: 14
loss: 7.758082866668701
epoch: 15
loss: 7.7524027824401855
epoch: 16
loss: 7.7467851638793945
epoch: 17
loss: 7.741224765777588
epoch: 18
loss: 7.735724449157715
epoch: 19
loss: 7.730283260345459
epoch: 20
loss: 7.7249016761779785
epoch: 21
loss: 7.719577312469482
epoch: 22
loss: 7.714314937591553
epoch: 23
loss: 7.70911169052124
epoch: 24
loss: 7.703969478607178
epoch: 25
loss: 7.6988844871521
epoch: 26
loss: 7.693858623504639
epoch: 27
loss: 7.688893795013428
epoch: 28
loss: 7.683988094329834
epoch: 29
loss: 7.6791

In [41]:
opt_lasso.score(X_test, y_test), opt_lasso_params

(0.7922897967423662, {'layer_sizes': [8, 8, 8, 8]})